In [1]:
# Importing libraries to work with
import re
import os
import nltk
import spacy
import torch
import string
import scipy as sci
import numpy as np
import pandas as pd
import gensim as gns
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
# Display the libraries' versions used in this notebook
version_list = {"NumPy Version:": np.__version__,
                "Pandas Version:": pd.__version__,
                "MatPlotLib Version:": mpl.__version__,
                "Seaborn Version:": sns.__version__,
                "PyTorch Version:": torch.__version__,
                "NLTK Version:": nltk.__version__,
                "SpaCy Version:": spacy.__version__,
                "Gensim Version:": gns.__version__,
                "SciPy Version:": sci.__version__}

for (k, v) in version_list.items():
    print(k, v)

NumPy Version: 1.26.4
Pandas Version: 2.2.3
MatPlotLib Version: 3.9.2
Seaborn Version: 0.13.2
PyTorch Version: 2.5.1+cpu
NLTK Version: 3.9.1
SpaCy Version: 3.8.2
Gensim Version: 4.3.3
SciPy Version: 1.13.1


In [ ]:
# Defining path to install NLTK libraries in
NLTK_LIB_PATH = ".\\venv_nlp\\Lib\\nltk_data"

try:
    os.mkdir(NLTK_LIB_PATH)
except FileExistsError:
    print("Directory Exists.")
except:
    print("Couldn't Make Directory.")

# Download extra parts of the library to use
try:
    nltk.data.find("tokenizers\\punkt.zip")                     # Punctuation
    print("Punctuation Data Exists.")
except LookupError:
    nltk.download('punkt', download_dir = NLTK_LIB_PATH)

try:
    nltk.data.find("corpora\\stopwords.zip")                    # Stopwords
    print("Stopwords Package Exists.")
except LookupError:
    nltk.download('stopwords', download_dir = NLTK_LIB_PATH)

try:
    nltk.data.find("corpora\\wordnet.zip")                      # Corpus
    print("Wordnet Package Exists.")
except LookupError:
    nltk.download('wordnet', download_dir = NLTK_LIB_PATH)

Directory Exists
Punctuation Data Exists
Stopwords Package Exists
Wordnet Package Exists


In [4]:
# Import the dependencies we've just loaded
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer